# Straka Bubble Test 🫧🫧🫧🫧🫧

<div class="alert alert-block alert-success">

## Part 5: Running Code in Terminal

All $\texttt{PADDLE}$ code can be run with terminal commands. The benefit of terminal, over Jupyter Notebook, is that simulations can be run with multiple cores (with options to use CPU *or* GPU, though we will stick with CPU for this bubble tutorial).

While the Straka Bubble Test is simple enough to run with one core in a Jupyter Notebook, when we move on to more complicated 3D atmospheric simulations, running code in terminal will become necessary. 

<div class="alert alert-info">

## Using Multiple CPU Cores

In order to use more than one CPU core, we will need to add a new dictionary entry to our `.yaml` file: $\texttt{distribute}$

### Define Distribute Properties ($\texttt{distribute}$)

The dictionary entry, $\texttt{distribute}$, describes how parallel computing is accomplished. Copy the following into your `.yaml` file: 

</div>

```yaml
distribute:
  backend: gloo
  layout: slab
  nb2: 2
  nb3: 1
  verbose: false

This is what the above means: 

In [ ]:
# Two choices for backend
# 'gloo' for CPU parallel computing
# 'nccl' for GPU
backend = 'gloo'

# Two choices for layout
# 'slab' for cartesian environments
# 'cubed-sphere' for 3D spheres
layout = 'slab'

# nb2 and nb3 are the number of blocks in each dimension
# These two numbers determine the amount of cores you can use
# For slab layouts, number of cores = nb2 * nb3
nb2 = 2
nb3 = 1

# verbose
# Running the code will always produce time-stepping outputs to terminal
# The verbose here is explicitly for printing out information useful for debugging
verbose = False

<div class="alert alert-info">

## Step-by-step running Straka in terminal


1. Open a terminal and activate your $\texttt{PADDLE}$ conda environment
2. `cd` into the directory storing the `straka.yaml` and `straka.py` (the directory with this notebook in it)
3. Ensure that `straka.yaml` has the distribute text labeled above
4. Run the following code in terminal (2 = number of cores = nb2 * nb1)
    ```
    torchrun --nproc-per-node=2 straka.py
    ```
4. Once finished, you'll notice a bundle of `.nc` files with `.out1` in them. This code combines them into a `main.nc` 
    ```
    pd-combine 1 -o main
    ```
5. Then to view
    ```
    ncview straka-main.nc
    ```

</div>

<div class="alert alert-warning">

Note that in the beta version of $\texttt{PADDLE}$, running things in terminal can sometimes be unstable. 

In particular, if an error occurs, its possible that sockets opened by using multiple cores will not be closed, which can cause a myriad of problems down the line. 

In order to manually close sockets, do the following in a terminal

`lsof -i:29501`

then

`pkill -9 python`

</div>

<div class="alert alert-block alert-success">

## $\texttt{PADDLE}$ Straka - Changing Input

Now that we are familiar with the Straka code, try changing the initial parameters! (The first cell in **Part 3**, or the portion in `straka.py` where these parameters are defined)

For example, try making a bubble with the following properties: 

Same pressure and temperature for background bubble

Ps = 1e5 

Ts = 300 

Larger bubble placed near center of simulation

xc = 13e3 

xr = 6e3 

zc = 4e3

zr = 3e3 

A bit colder 

dT = -20

**To not overwrite your first bubble's results, make sure to name your `.yaml` file something different, maybe `straka_2.yaml`, since the output file names will be based on that!**

</div>

<div class="alert alert-info">

## Other Key Differences in `.py` script not found in Jupyter Notebook 

While minor, here we document other differences that you will find in the `.py` script version of Straka than you will find in how we made the executable code step-by-step in Part 3. 

Mainly, this portion of the code helps the code know whether or not you are running on CPUs or GPUs, as both are compatible with $\texttt{PADDLE}$

First, you will note that there is this line of code that sets a variable called `device`. This code just detects whether or not you are using a CPU or a GPU . 

```python 
# use cuda if available
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

When initializing the MeshBlock, we let it know about which device is being used. 

```python 
# initialize block
block = MeshBlock(op)
block.to(device)

Finally, we also let the torch tensor know about the device when we initialize it. In the Jupyter notebook, we had just manually set device to `cpu`.

```python 
w = torch.zeros((nvar, nc3, nc2, nc1), device=device)